In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

def define_model(path_to_model):
    tokenizer = AutoTokenizer.from_pretrained(path_to_model)
    model = AutoModelForCausalLM.from_pretrained(path_to_model,
                                                 torch_dtype="auto",
                                                 device_map="auto")
    return tokenizer, model

In [ ]:
# Инициализация учителя

teacher_tokenizer, teacher_model = define_model('./teacher_model/')

In [ ]:
# Инициализация few-shot'a

few_shot = '''
Problem:
Olivia has $23. She bought five cupcakes for $3 each and a milkshake for $4. How much money does she have left?
Solution:
First, calculate the total cost of the cupcakes:
5 cupcakes × $3 = $15.
Add the cost of the milkshake: $15 + $4 = $19.
Subtract from her initial amount: $23 − $19 = $4.
So, Olivia has $4 left.
Final Answer: 4​

Problem:
A bakery sells cookies in packs of 6. If a customer buys 9 packs, how many cookies does the customer get in total?
Solution:
Each pack contains 6 cookies.
The customer buys 9 packs.
Total cookies = 6 × 9 = 54.
Final Answer: 54

Problem:
There are 42 students in a class. One-third of them are boys. How many girls are in the class?
Solution:
Number of boys = 42 ÷ 3 = 14.
Number of girls = total students − boys = 42 − 14 = 28.
Final Answer: 28

Problem:
A car travels 60 miles per hour. How many miles does it travel in 2 hours and 30 minutes?
Solution:
Convert 2 hours 30 minutes to hours: 2.5 hours.
Distance = speed × time = 60 × 2.5 = 150 miles.
Final Answer: 150

Problem:
James has 3 times as many marbles as Lisa. Together, they have 48 marbles. How many marbles does James have?

Solution:
Let Lisa have x marbles.
Then James has 3x marbles.
Together: x + 3x = 4x = 48.
So, x = 48 ÷ 4 = 12.
James has 3 × 12 = 36 marbles.
Final Answer: 36
'''

In [ ]:
# Функция генерации ответа на запрос

def invoke_llm(type_of_model, task, few_shot=few_shot):

    if type_of_model == 'teacher':
        tokenizer = teacher_tokenizer
        model = teacher_model
    else:
        return None
    
    messages = [{"role": "user", 
                 "content": few_shot + '\n' + task}]
    
    text = tokenizer.apply_chat_template(messages,
                                         tokenize=False,
                                         add_generation_prompt=True,
                                         enable_thinking=False)
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
    return content

In [ ]:
# Импорт данных

import polars as pl

splits = {'train': 'main/train-00000-of-00001.parquet'}
df_train = pl.read_parquet("hf://datasets/openai/gsm8k/" + splits["train"])

questions = df_train['question'].to_list()
answers = df_train['answer'].to_list()

In [ ]:
# Генерация по таскам из GSM8K

import time
from tqdm import tqdm

with open(f'full_train_data_1.txt', 'a', encoding='utf-8', buffering=1) as f:
    for q, a in tqdm(zip(questions, answers), total=len(questions)):
        start_time = time.time()
        teacher_response = invoke_llm('teacher', q)
        f.write(teacher_response + '\n')
        f.write(f'-----'*20 + '\n')
        f.flush()
        end_time = time.time()
        elapsed_time = end_time - start_time